In [9]:
import pandas as pd
from newsapi import NewsApiClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

# Inicialize a API do NewsAPI
newsapi = NewsApiClient(api_key='3be6fb74511c432bb6741415eb0419d7')

# Função para obter notícias
def get_stock_news(ticker, from_date, to_date, page_size=100, page=1):
    all_articles = newsapi.get_everything(q=ticker,
                                          from_param=from_date,
                                          to=to_date,
                                          language='pt',
                                          sort_by='relevancy',
                                          page_size=page_size,
                                          page=page)
    return all_articles['articles']

# Função para classificar o sentimento
def classify_sentiment(text, analyzer):
    scores = analyzer.polarity_scores(text)
    if scores['compound'] >= 0.05:
        return 'positivo'
    elif scores['compound'] <= -0.05:
        return 'negativo'
    else:
        return 'neutro'

if __name__ == "__main__":
    ticker = 'VIVT3'
    articles = []
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)
    
    # Obtenha notícias dos últimos 30 dias
    for i in range(0, 30, 5):
        from_date = (start_date + timedelta(days=i)).strftime('%Y-%m-%d')
        to_date = (start_date + timedelta(days=i + 5)).strftime('%Y-%m-%d')
        for page in range(1, 2):
            articles.extend(get_stock_news(ticker, from_date, to_date, page_size=100, page=page))
    
    # Crie um DataFrame com as notícias
    df = pd.DataFrame(articles)
    
    # Inicialize o analisador de sentimentos
    analyzer = SentimentIntensityAnalyzer()
    
    # Classifique o sentimento das notícias
    df['sentiment'] = df['content'].apply(lambda x: classify_sentiment(x, analyzer) if x else 'neutro')
    
    # Exiba os resultados da análise de sentimentos
    print(df['sentiment'].value_counts())
    print(df[['title', 'sentiment']])

sentiment
neutro      18
negativo     7
Name: count, dtype: int64
                                                title sentiment
0   Internet ficará mais cara após medida protecio...  negativo
1   Telefônica Vivo (VIVT3) registra lucro 13% sup...    neutro
2   Vivo (VIVT3) convocará AGE para votar restitui...  negativo
3   Ações de dividendos para novembro: Vibra (VBBR...    neutro
4   Eleições nos EUA, balanço da Gerdau, nova reun...    neutro
5   Vivo, GPA, PRIO, RD, Vibra, Odontoprev, Marisa...    neutro
6   Ibovespa tem alta discreta de 0,11% à espera d...    neutro
7   Por que esta é a semana mais importante de 202...    neutro
8   Veja 4 ações para lucrar em uma economia bagun...    neutro
9   Vivo pode embolsar R$ 4,5 bi com transição reg...  negativo
10  TIM (TIMS3): estratégia de preços mais agressi...  negativo
11  JPMorgan vê planos da NuCel mais caros e reite...    neutro
12  Veja operações de day trade da Embraer e Vivo ...    neutro
13  TCU aprova acordo de solução conse

In [8]:
#Verifique valores distintos
df['sentiment'].unique()


array(['negativo', 'neutro'], dtype=object)

In [ ]:
import pandas as pd
from newsapi import NewsApiClient
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from datetime import datetime, timedelta

# Inicialize a API do NewsAPI
newsapi = NewsApiClient(api_key='3be6fb74511c432bb6741415eb0419d7')

# Função para obter notícias
def get_stock_news(ticker, from_date, to_date, page_size=100, page=1):
    all_articles = newsapi.get_everything(q=ticker,
                                          from_param=from_date,
                                          to=to_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=page_size,
                                          page=page)
    return all_articles['articles']

# Inicialize o modelo FinBERT
model_name = "yiyanghkust/finbert-tone"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Função para classificar o sentimento usando FinBERT
def classify_sentiment(text):
    nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
    result = nlp(text)
    return result[0]['label']

if __name__ == "__main__":
    ticker = 'AAPL'
    articles = []
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)
    
    # Obtenha notícias dos últimos 30 dias
    for i in range(0, 30, 5):
        from_date = (start_date + timedelta(days=i)).strftime('%Y-%m-%d')
        to_date = (start_date + timedelta(days=i + 5)).strftime('%Y-%m-%d')
        for page in range(1, 2):
            articles.extend(get_stock_news(ticker, from_date, to_date, page_size=100, page=page))
    
    
    df = pd.DataFrame(articles)
    
    # Classifique o sentimento das notícias
    df['sentiment'] = df['description'].apply(lambda x: classify_sentiment(x) if x else 'neutro')
    
    # Exiba os resultados da análise de sentimentos
    print(df['sentiment'].value_counts())
    print(df[['title', 'sentiment']])

sentiment
Neutral     332
Positive     69
Negative     35
neutro        2
Name: count, dtype: int64
                                                 title sentiment
0    Market Clubhouse Morning Memo - November 5th, ...   Neutral
1    Warren Buffett's Berkshire Hathaway has enough...  Positive
2    Nvidia and Google stock rise while Meta and TS...   Neutral
3    Warren Buffett’s Berkshire Hathaway is selling...  Positive
4    Nvidia and Apple keep taking turns at the top ...  Positive
..                                                 ...       ...
433  Apple (NASDAQ:AAPL) Reaches New 12-Month High ...   Neutral
434  Cadinha & Co. LLC Has $5.30 Million Holdings i...   Neutral
435           Sunday Summary: No Rest for the Holidays   Neutral
436  Purpose Investments Inc. Announces 2024 Estima...   Neutral
437  Purpose Investments Inc. Announces 2024 Estima...   Neutral

[438 rows x 2 columns]
